In [3]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime, timedelta

## Load data function

In [4]:
def read_csv_file(stock, date):
    year = datetime.strptime(date, "%Y%m%d").year

    # Define possible base directories
    base_dirs = [
        f"{stock}/{year}/{year}_new/",
        f"{stock}/{year}/",
        f"{stock}/{year}_new/"
    ]

    filename_pattern = f"{stock}_{date}_*_Intraday_Preprocessed.csv"

    for base_dir in base_dirs:
        search_pattern = os.path.join(base_dir, filename_pattern)
        matching_files = glob.glob(search_pattern)

        if matching_files:
            # Load the first matching file
            return pd.read_csv(matching_files[0])

    print("File not found in any of the expected locations:")
    for base_dir in base_dirs:
        print(f"  {os.path.join(base_dir, filename_pattern)}")

    return None

In [ ]:
def build_yearly_frames(stock):
    """
    Builds 12 expiry-cycle frames from intraday option data files using pandas.

    Filters out rows where:
    - price_problem == True
    - is_tradable == False

    Returns:
        A list of pandas DataFrames.
    """
    base_path = f"{stock}/2024/2024_new/"
    pattern = os.path.join(base_path, f"{stock}_*_Intraday_Preprocessed.csv")
    file_paths = sorted(glob.glob(pattern))

    # Extract date pairs (start_date, expiry_date)
    date_pairs = []
    for path in file_paths:
        fname = os.path.basename(path)
        try:
            parts = fname.split('_')
            start_date = parts[1]
            expiry_date = parts[2]
            date_pairs.append((start_date, expiry_date, path))
        except IndexError:
            continue

    date_pairs.sort(key=lambda x: x[0])

    frames = []
    used_indices = set()
    current_index = 0

    while len(frames) < 12 and current_index < len(date_pairs):
        frame_paths = []
        start_date, expiry_date, _ = date_pairs[current_index]
        frame_start = start_date
        frame_expiry = expiry_date

        # Collect paths for current expiry cycle
        for i in range(current_index, len(date_pairs)):
            s_date, e_date, path = date_pairs[i]
            if s_date >= frame_start and s_date <= frame_expiry:
                frame_paths.append(path)
                used_indices.add(i)

        if frame_paths:
            dfs = []
            for path in frame_paths:
                df = pd.read_csv(path)

                # Filter: keep only rows where price_problem == False and is_tradable == True
                if "price_problem" in df.columns and "is_tradable" in df.columns:
                    df = df[(df["price_problem"] == False) & (df["is_tradable"] == True)]

                dfs.append(df)

            if dfs:
                frame_df = pd.concat(dfs, axis=0, ignore_index=True)
                frames.append(frame_df)

        # Move to next frame: after expiry
        next_start_date = (datetime.strptime(frame_expiry, "%Y%m%d") + timedelta(days=1)).strftime("%Y%m%d")
        found_next = False
        for i in range(current_index + 1, len(date_pairs)):
            if i in used_indices:
                continue
            if date_pairs[i][0] >= next_start_date:
                current_index = i
                found_next = True
                break

        if not found_next:
            break

    return frames

In [ ]:
def process_frame(df: pd.DataFrame, lot_size: int, weight: float, h: int):
    """
    - Sells 1 ATM straddle at bid price on first Date Time
    - Delta hedges using weight-scaled underlying trades every h minutes
    - Unwinds at 15:30 on expiry day using intrinsic value
    - Returns net PnL of the strategy
    """
    df = df.sort_values("Date Time")
    df["Date Time"] = pd.to_datetime(df["Date Time"]) 
    
    first_ts = df["Date Time"].min()
    
    first_min_df = df[df["Date Time"] == first_ts]
    spot_price = first_min_df["Spot"].mean()
    
    # Find ATM strike with both CE and PE
    available_strikes = sorted(first_min_df["Strike"].unique(), key=lambda x: abs(x - spot_price))
    
    def has_both_legs(strike_val):
        sub = first_min_df[first_min_df["Strike"] == strike_val]
        types = sub["Type"].unique()
        return ("CE" in types) and ("PE" in types)
    
    valid_strike = next((s for s in available_strikes if has_both_legs(s)), None)
    if valid_strike is None:
        raise ValueError("No strike with both CE and PE at entry.")
    
    # Sell lot_size * 1 ATM straddle at bid price
    atm_straddle = first_min_df[(first_min_df["Strike"] == valid_strike) & (first_min_df["Type"].isin(["CE", "PE"]))]
    ce_bid = atm_straddle[atm_straddle["Type"] == "CE"]["BidPrice"].values[0]
    pe_bid = atm_straddle[atm_straddle["Type"] == "PE"]["BidPrice"].values[0]
    straddle_entry_price = ce_bid + pe_bid
    premium_received = straddle_entry_price * lot_size
    
    # Build list of hedge timestamps
    timestamps = sorted(pd.to_datetime(df["Date Time"]).unique())
    timestamps = [pd.Timestamp(ts).to_pydatetime() for ts in timestamps]  # all are datetime.datetime now
    expiry_ts = pd.Timestamp(max(timestamps)).to_pydatetime()
    expiry_cutoff = expiry_ts.replace(hour=15, minute=30, second=0, microsecond=0)
    
    hedge_times = []
    current_time = first_ts
    while current_time < expiry_cutoff:
        day = current_time.date()
        day_end = datetime.combine(day, datetime.min.time()).replace(hour=15, minute=30)
        while current_time < day_end and current_time < expiry_cutoff:
            if current_time in timestamps:
                hedge_times.append(current_time)
            current_time += timedelta(minutes=h)
        next_day = day + timedelta(days=1)
        next_day_start = datetime.combine(next_day, datetime.min.time()).replace(hour=9, minute=15)
        future_ts = [ts for ts in timestamps if ts >= next_day_start]
        if not future_ts:
            break
        current_time = future_ts[0]
    
    # Portfolio tracking
    spot_position = 0.0
    cash = premium_received
    
    for hedge_time in hedge_times:
        snap = df[df["Date Time"] == hedge_time]
        ce_row = snap[(snap["Strike"] == valid_strike) & (snap["Type"] == "CE")]
        pe_row = snap[(snap["Strike"] == valid_strike) & (snap["Type"] == "PE")]
        if ce_row.empty or pe_row.empty:
            continue
        ce_delta = ce_row["Delta"].values[0]
        pe_delta = pe_row["Delta"].values[0]
        total_delta = (ce_delta + pe_delta) * lot_size
        net_delta = weight * total_delta
        
        spot = snap["Spot"].values[0]
        hedge_trade = -net_delta
        cash -= hedge_trade * spot
        spot_position += hedge_trade
    
    # Unwind at expiry 15:30
    final_df = df[df["Date Time"] == expiry_cutoff]
    final_spot = final_df["Spot"].values[0]
    intrinsic_ce = max(0, final_spot - valid_strike)
    intrinsic_pe = max(0, valid_strike - final_spot)
    straddle_intrinsic = intrinsic_ce + intrinsic_pe
    final_value = straddle_intrinsic * lot_size
    
    cash += spot_position * final_spot
    pnl = cash - final_value
    
    return pnl

In [7]:
stocks = ["ASIANPAINT", "BAJAJ-AUTO"] # Available "ASIANPAINT", "BAJAJ-AUTO", "BAJFINANCE", "HDFCBANK", "ICICIBANK", "RELIANCE", "SBIN", "TATAMOTORS", "TCS", "TITAN"
weight = [1/len(stocks)] * len(stocks) # weights for the assets

# Lot_sizes
df = pd.read_csv('NIFTY_200_Lot_Size.csv')
df['Symbol'] = df['Symbol'].str.strip().str.upper()
df['LOT_SIZE'] = pd.to_numeric(df['LOT_SIZE'], errors='coerce')

lot_sizes = []

for symbol in stocks:
    symbol = symbol.strip().upper()
    lot_size_row = df[df['Symbol'] == symbol]

    if not lot_size_row.empty:
        lot_size = lot_size_row['LOT_SIZE'].values[0]
    else:
        lot_size = None

    lot_sizes.append(lot_size)

In [8]:
s_1 = build_yearly_frames(stocks[0])

In [9]:
pnl = process_frame(s_1[0], lot_sizes[0], weight[0], 5)  # Example with first stock and its lot size

In [10]:
pnl

-16252006.402672112

In [12]:
s_1[0]

,Date Time,ExchToken,BidPrice,BidQty,AskPrice,AskQty,TTq,LTP,TotalTradedPrice,Instrument,...,Vega,Sigma,bid_ask_spread,mid_price,Intrinsic_value,bid_ask_move,price_problem,is_tradable,bid_plus,ask_minus
0,2024-01-01 09:16:00,63547,760.65,8000.0,967.10,8000.0,0.0,0.0,0.0,ASIANPAINT,...,0.115141,0.441593,206.45,863.875,863.67,0.238981,False,True,812.2625,915.4875
1,2024-01-01 09:16:00,63549,780.50,600.0,926.10,600.0,0.0,0.0,0.0,ASIANPAINT,...,0.777531,0.665762,145.60,853.300,843.67,0.170632,False,True,816.9000,889.7000
2,2024-01-01 09:16:00,63551,760.80,600.0,906.10,600.0,0.0,0.0,0.0,ASIANPAINT,...,0.790200,0.650533,145.30,833.450,823.67,0.174336,False,True,797.1250,869.7750
3,2024-01-01 09:16:00,63569,740.80,600.0,886.10,600.0,0.0,0.0,0.0,ASIANPAINT,...,0.803272,0.635386,145.30,813.450,803.67,0.178622,False,True,777.1250,849.7750
4,2024-01-01 09:16:00,63571,720.70,600.0,866.10,600.0,0.0,0.0,0.0,ASIANPAINT,...,0.816768,0.620317,145.40,793.400,783.67,0.183262,False,True,757.0500,829.7500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
666265,2024-01-25 15:30:00,65945,553.55,1400.0,604.40,400.0,0.0,0.0,0.0,ASIANPAINT,...,0.010322,33.801241,50.85,578.975,570.67,0.087828,False,True,566.2625,591.6875
666266,2024-01-25 15:30:00,65955,575.95,1400.0,621.75,600.0,0.0,0.0,0.0,ASIANPAINT,...,0.010156,34.644786,45.80,598.850,590.67,0.076480,False,True,587.4000,610.3000
666267,2024-01-25 15:30:00,65957,593.80,1400.0,636.70,200.0,200.0,58370.0,11674000.0,ASIANPAINT,...,0.007237,31.990261,42.90,615.250,610.67,0.069728,False,True,604.5250,625.9750
666268,2024-01-25 15:30:00,65963,613.90,1400.0,660.45,400.0,0.0,0.0,0.0,ASIANPAINT,...,0.009027,35.243619,46.55,637.175,630.67,0.073057,False,True,625.5375,648.8125
